<a href="https://colab.research.google.com/github/pvai-umich/SIADS697/blob/main/Data/Load_Kaggle_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data

This notebook will contain the data loading scripts for the two kaggle data sets.

Set up the Kaggle login and API key:

In [ ]:
import os
import zipfile
import pandas as pd

#comment

os.environ['KAGGLE_USERNAME'] = "pvaiciunas" # username from the json file
os.environ['KAGGLE_KEY'] = "fba13e65a047582a90e611238731617e" # key from the json file

## Kaggle Indeed Jobs Dataset

Source:
https://www.kaggle.com/promptcloud/indeed-job-posting-dataset

The next code block will download the zip file, unzip, and read the csv file into a pandas dataframe.

In [ ]:
!kaggle datasets download -d promptcloud/indeed-job-posting-dataset # api copied from kaggle

with zipfile.ZipFile('indeed-job-posting-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

dat_indeed = pd.read_csv("home/sdf/marketing_sample_for_trulia_com-real_estate__20190901_20191031__30k_data.csv")


## Kaggle Monster Jobs Dataset

Source:
https://www.kaggle.com/PromptCloudHQ/us-jobs-on-monstercom

The next code block will download the zip file, unzip, and read the csv file into a pandas dataframe.

In [ ]:
!kaggle datasets download -d PromptCloudHQ/us-jobs-on-monstercom

with zipfile.ZipFile('us-jobs-on-monstercom.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

dat_monster = pd.read_csv("monster_com-job_sample.csv")

home				sample_data
indeed-job-posting-dataset.zip	us-jobs-on-monstercom.zip
monster_com-job_sample.csv
